<a href="https://colab.research.google.com/github/jyyyyyeeeee/autonomous-driving-captioning/blob/main/InstructBLIP_Inference_after_Fine_tuning_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 의존성 설치 (훈련 때와 동일 버전)

In [1]:
!pip uninstall -y numpy torch torchvision torchaudio transformers accelerate peft bitsandbytes datasets xformers -q
!pip install -q "numpy==2.0.2"
!pip install -q --index-url https://download.pytorch.org/whl/cu121 \
  torch==2.3.1 torchvision==0.18.1 torchaudio==2.3.1
!pip install -q transformers==4.41.2 peft==0.11.1 datasets==2.19.2 tokenizers==0.19.1 bitsandbytes==0.43.1 accelerate==0.30.1
!pip install -q sentencepiece==0.1.99 timm==0.9.16


ERROR: Operation cancelled by user
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 110.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.17.0 requires torch>=1.13.0, which is not installed.
accelerate 1.10.0 requires torch>=2.0.0, which is not installed.
fastai 2.7.19 requires torch<2.7,>=1.10, which is not installed.
torchvision 0.21.0+cu124 requires torch==2.6.0, which is not installed.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 781.0/781.0 MB 207.7 MB/s eta 0:00:01
ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/bin/pip3", line 4, in <module>
    from pip._internal.cli.main import main
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/main.py", line 11, in <module>
    from pip._internal.cli

프로세서/토크나이저 (훈련과 동일한 구성)

In [2]:
import torch
from transformers import AutoTokenizer, AutoImageProcessor, InstructBlipProcessor

MODEL_NAME = "Salesforce/instructblip-vicuna-7b"

# Vicuna/LLaMA 토크나이저(slow 권장)
txt_tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)

# Q-Former용 BERT 토크나이저
qf_tok  = AutoTokenizer.from_pretrained("bert-base-uncased")

# 이미지 프로세서
img_proc = AutoImageProcessor.from_pretrained(MODEL_NAME)

# 하나의 Processor로 조립
processor = InstructBlipProcessor(
    image_processor=img_proc,
    tokenizer=txt_tok,
    qformer_tokenizer=qf_tok,
)

tokenizer = processor.tokenizer
if tokenizer.pad_token_id is None and tokenizer.eos_token_id is not None:
    tokenizer.pad_token_id = tokenizer.eos_token_id

print("OK. fast tokenizer?", getattr(tokenizer, "is_fast", False))


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/549 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/439 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


OK. fast tokenizer? False


베이스 모델 로드 + LoRA 어댑터 적용

In [3]:
import os, glob
from peft import PeftModel
from transformers import InstructBlipForConditionalGeneration

bf16 = torch.cuda.is_available() and torch.cuda.get_device_capability(0)[0] >= 8
dtype = torch.bfloat16 if bf16 else torch.float16

# 8bit 로드 (메모리 부족 시 load_in_4bit=True로 전환 가능)
base_model = InstructBlipForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    torch_dtype=dtype,
    load_in_8bit=True,
)

RUN_DIR = "/content/drive/MyDrive/instructblip_lora_run1"
ckpts = sorted(glob.glob(os.path.join(RUN_DIR, "checkpoint-*")), key=lambda p: int(p.split("-")[-1]))
assert ckpts, f"체크포인트가 없습니다: {RUN_DIR}"
adapter_path = ckpts[-1]
print("Using adapter:", adapter_path)

model = PeftModel.from_pretrained(base_model, adapter_path)
model.to("cuda")
model.eval()


PackageNotFoundError: No package metadata was found for torch

In [ ]:
from PIL import Image

IMAGE_PATH = "/content/drive/MyDrive/2025urop-captioning/추론용 이미지/b77dc345-d1f9e790.jpg"
question = (
    "Answer in exactly 2 short sentences.\n"
    "Use lowercase 'none' if not visible.\n"
    "Sentence 1: Car lights color: <colors or none>.\n"
    "Sentence 2: Traffic light color: <red/green/yellow or none>."
)

prompt = f"USER: {question}\nASSISTANT: "

image = Image.open(IMAGE_PATH).convert("RGB")
enc = processor(images=image, text=prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
enc = {k: v.to(model.device) for k, v in enc.items()}

with torch.inference_mode():
 out_ids = model.generate(
    **enc,
    max_new_tokens=140,         # 끊김 방지 여유
    do_sample=False,            # 우선 결정론적으로 안정성 확인
    num_beams=4,                # 완결성↑ (반복 경향 ↓)
    no_repeat_ngram_size=4,     # 3-그램 이상 반복 차단
    repetition_penalty=1.25,     # 동일/유사 문장 반복 억제
    length_penalty=0.8,         # 장황해지는 것 방지
    early_stopping=True,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.pad_token_id,
    remove_invalid_values=True
)


decoded = tokenizer.decode(out_ids[0], skip_special_tokens=True)
answer = decoded.split("ASSISTANT:")[-1].strip()
print("\n=== PREDICTION ===\n", answer)


In [ ]:
import re
import torch
from transformers import StoppingCriteria, StoppingCriteriaList
from PIL import Image

# (필수) processor/tokenizer/model 은 이미 로드돼 있다고 가정
# - model  : PeftModel(InstructBlipForConditionalGeneration+LoRA)
# - processor: InstructBlipProcessor(...)
# 아래 한 줄로 tokenizer 핸들만 확실히 잡아둠
tokenizer = processor.tokenizer

# 1) 입력 준비
IMAGE_PATH = "/content/drive/MyDrive/2025urop-captioning/추론용 이미지/d33f0c00-a05ea7e1.jpg"
question = "Describe the traffic scene for an autonomous driving agent.\n"
prompt = f"USER: {question}\nASSISTANT: "

image = Image.open(IMAGE_PATH).convert("RGB")
enc = processor(images=image, text=prompt, return_tensors="pt",
                padding=True, truncation=True, max_length=512)
enc = {k: v.to(model.device) for k, v in enc.items()}

# (경고 억제) max_length 기본값과 max_new_tokens 충돌 방지
model.generation_config.max_length = None

# 2) 문장 수 기반 스톱: 마침표/물음표/느낌표로 끝난 문장 n개가 생성되면 중단
class StopOnNSentences(StoppingCriteria):
    def __init__(self, tok, start_len, n=5):
        self.tok = tok
        self.start_len = start_len
        self.n = n
    def __call__(self, input_ids, scores, **kwargs):
        txt = self.tok.decode(input_ids[0, self.start_len:], skip_special_tokens=True)
        # 문장 끝나는 구두점 기준으로만 카운트 → 미완성 문장 중간에 안 끊김
        sents = re.findall(r'[^.!?]+[.!?]', txt)
        return len(sents) >= self.n

start_len = enc["input_ids"].shape[1]
stops = StoppingCriteriaList([StopOnNSentences(tokenizer, start_len, n=5)])  # 최대 5문장

# 3) 생성 (반복/모순 완화용 디코딩 설정)
with torch.inference_mode():
    out = model.generate(
        **enc,
        do_sample=False,          # 재현성
        num_beams=3,              # 완결성↑, 헛말↓
        max_new_tokens=90,        # 장황 방지 (필요시 80~120 범위 조정)
        no_repeat_ngram_size=4,   # 구문 반복 억제
        repetition_penalty=1.2,   # 의미 반복 억제
        length_penalty=0.9,       # 불필요한 늘이기 방지
        early_stopping=True,
        stopping_criteria=stops,  # 문장 수 제한(후처리 아님, 디코딩 일부)
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
        remove_invalid_values=True,
        return_dict_in_generate=True,
    )

# 4) 프롬프트 부분 제외하고 '생성된 토큰'만 디코딩 (에코 제거)
gen_ids = out.sequences[:, start_len:]
answer = tokenizer.decode(gen_ids[0], skip_special_tokens=True).strip()

print("\n=== PREDICTION ===\n", answer)

In [ ]:
from PIL import Image

IMAGE_PATH = "/content/drive/MyDrive/2025urop-captioning/추론용 이미지/d33f0c00-a05ea7e1.jpg"
question = (
    "Describe the traffic scene for an autonomous driving agent."
)

prompt = f"USER: {question}\nASSISTANT: "

image = Image.open(IMAGE_PATH).convert("RGB")
enc = processor(images=image, text=prompt, return_tensors="pt", padding=True, truncation=True, max_length=512)
enc = {k: v.to(model.device) for k, v in enc.items()}

with torch.inference_mode():
    out_ids = model.generate(
        **enc,
        max_new_tokens=128,
        do_sample=False,        # 우선 그리디로 확인
        num_beams=1,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id
    )

decoded = tokenizer.decode(out_ids[0], skip_special_tokens=True)
answer = decoded.split("ASSISTANT:")[-1].strip()
print("\n=== PREDICTION ===\n", answer)

이거!

In [ ]:
from PIL import Image

IMAGE_PATH = "/content/drive/MyDrive/2025urop-captioning/추론용 이미지/d33f0c00-a05ea7e1.jpg"
question = "Describe the traffic scene for an autonomous driving agent."
prompt = f"USER: {question}\nASSISTANT: "

image = Image.open(IMAGE_PATH).convert("RGB")
enc = processor(images=image, text=prompt, return_tensors="pt",
                padding=True, truncation=True, max_length=512)
enc = {k: v.to(model.device) for k, v in enc.items()}

# 경고 방지
model.generation_config.max_length = None

with torch.inference_mode():
    out_ids = model.generate(
        **enc,
        do_sample=False,
        num_beams=3,
        max_new_tokens=100,
        no_repeat_ngram_size=4,
        repetition_penalty=1.15,
        early_stopping=True,
        length_penalty=1.0,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id=tokenizer.pad_token_id,
    )

decoded = tokenizer.decode(out_ids[0], skip_special_tokens=True)
answer = decoded.split("ASSISTANT:")[-1].strip()
print("\n=== PREDICTION ===\n", answer)


In [ ]:
import os
import csv
from PIL import Image
from tqdm import tqdm

# ===== 경로 =====
IMAGE_DIR = "/content/drive/MyDrive/2025urop-captioning/추론용 이미지 100장"
OUT_CSV   = "/content/drive/MyDrive/2025urop-captioning/finetuned_predictions.csv"

question = "Describe the traffic scene for an autonomous driving agent."
prompt = f"USER: {question}\nASSISTANT: "

def is_image(fname):
    return fname.lower().endswith((".jpg", ".jpeg", ".png", ".webp"))

files = [f for f in os.listdir(IMAGE_DIR) if is_image(f)]

with open(OUT_CSV, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(["image_id", "caption"])

    for fname in tqdm(files, desc="Finetuned captioning"):
        image_path = os.path.join(IMAGE_DIR, fname)
        image = Image.open(image_path).convert("RGB")

        enc = processor(images=image, text=prompt, return_tensors="pt",
                        padding=True, truncation=True, max_length=512)
        enc = {k: v.to(model.device) for k, v in enc.items()}

        # 경고 방지
        model.generation_config.max_length = None

        with torch.inference_mode():
            out_ids = model.generate(
                **enc,
                do_sample=False,
                num_beams=3,
                max_new_tokens=100,
                no_repeat_ngram_size=4,
                repetition_penalty=1.15,
                early_stopping=True,
                length_penalty=1.0,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id,
            )

        decoded = tokenizer.decode(out_ids[0], skip_special_tokens=True)
        caption = decoded.split("ASSISTANT:")[-1].strip()

        writer.writerow([fname, caption])

print(f"✅ 완료: {OUT_CSV}")
